# Get started with Zawadi Tunu Built with Memory Bank on ADK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/mwanyumba7/Zawadi-Tunu/blob/main/zawadi_tunu_with_memory_bank_on_adk.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
</table>

## Overview
We will be building an Agent `Zawadi Tunu` which is an agent ispecialized in gift recommendation engine designed to elevate the act of gift-giving, transforming it from a chore into a deeply personalized and meaningful experience. Built as a demonstration of the Agent Development Kit (ADK) for Gemini and the Vertex AI Memory Bank, this agent's core strength lies in its ability to retain and recall nuanced information over time.

Instead of generic suggestions, the agent uses its long-term memory to remember key details gleaned from previous conversations: a niece's age and past birthday presents (like the "red bike" in the demo), a spouse's stated preference for aisle seats, or a parent's hint about a desired hobby.

This tutorial demonstrates how to build ADK agents with Memory, including both `InMemoryMemoryService` and `VertexAiMemoryBankService`.

This tutorial will cover:
* Comparing `InMemoryMemoryService` vs. `VertexAiMemoryBankService`.
* Generating memories with ADK and Agent Engine Memory Bank.
* Retrieving memories with ADK and Agent Engine Memory Bank.
* Customizing your Memory Bank instance's behavior.

## Get started

### Install Google Gen AI SDK and other required packages


In [99]:
%pip install google-adk google-cloud-aiplatform --upgrade --quiet

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [100]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [101]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

# fmt: off
PROJECT = "data-bwai"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT or PROJECT == "[your-project-id]":
    PROJECT = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = vertexai.Client(project=PROJECT, location=LOCATION)

# Set environment variables for ADK.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

## (Optional) Set up logging
To surface ADK logs in your notebook, you may need to configure the logging level of ADK's logger.

In [102]:
import logging
import sys

# Set up a general handler for all logs to be printed in your Colab output.
# We'll set the overall level to INFO.
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    stream=sys.stdout,
)

# Get the specific logger used by ADK.
adk_logger = logging.getLogger("google_adk")

# Set its level to DEBUG to ensure you capture ALL messages from the SDK,
# even if the general level is higher (like INFO).
adk_logger.setLevel(logging.DEBUG)

## Define a simple agent

The ADK Runner defines what session service is used by the agent. The session service is responsible for saving the conversation history.

In this example, we're using the in-memory session service (`InMemorySessionService`). However, you can use any session service with Memory.

If you want to use Google's pre-built Memory tools, you can provide a Memory service to the Runner as well. In this example, we'll start by directly invoking memory so that we can compare different services. So, we're not directly setting `memory_service` in the Runner.

In [103]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

APP_NAME = "memory_example_app"
MODEL = "gemini-2.5-flash"

agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
)

session_service = InMemorySessionService()

runner = Runner(agent=agent, app_name=APP_NAME, session_service=session_service)


def call_agent(query, session, user_id):
    content = Content(role="user", parts=[Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

## Introduction to ADK Memory Service
ADK memory services offer an interface for creating and searching for memories. The [`BaseMemoryService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/base_memory_service.py) defines the interface for managing this searchable, long-term knowledge store. Its primary responsibilities are:

* Ingesting Information (`add_session_to_memory`): Taking the contents of a (usually completed) Session and adding relevant information to the long-term knowledge store.
* Searching Information (`search_memory`): Allowing an agent (typically via a Tool) to query the knowledge store and retrieve relevant snippets or context based on a search query.

In this colab, we'll cover two options:

* `InMemoryMemoryService`: A in-memory service that allows you to search your conversation history to retrieve the raw dialogue using basic keyword matching.
* `VertexAiMemoryBankService`: A remote service that remotely generates memories for "meaningful" information from user conversation.


You can also interact directly with Memory Bank by using the Agent Engine SDK.

### `InMemoryMemoryService`

[`InMemoryMemoryService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/in_memory_memory_service.py) persists **all** information in-memory. The events are not condensed, and information is not extracted from them.

In [104]:
from google.adk.memory import InMemoryMemoryService

in_memory_service = InMemoryMemoryService()

### `VertexAiMemoryBankService`
Memory Bank will only persist information that's meaningful for future interactions. [`VertexAiMemoryBankService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/vertex_ai_memory_bank_service.py) provides a built-in integration between Memory Bank and ADK.

To get started with Memory Bank, you need to first create an Agent Engine. This only takes a few seconds.

If you don't have a Google Cloud project, you can use [Vertex in Express mode, which only requires an API key](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/set-up#authentication).

In [105]:
import vertexai
from google.adk.memory import VertexAiMemoryBankService

client = vertexai.Client(project=PROJECT, location=LOCATION)

agent_engine = client.agent_engines.create(
    config={
        "context_spec": {
            "memory_bank_config": {
                "generation_config": {
                    "model": f"projects/{PROJECT}/locations/{LOCATION}/publishers/google/models/gemini-2.5-flash"
                }
            }
        }
    }
)

memory_bank_service = VertexAiMemoryBankService(
    agent_engine_id=agent_engine.api_resource.name.split("/")[-1],
    project=PROJECT,
    location=LOCATION,
)

INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=data-bwai.
INFO:vertexai_genai.agentengines:Agent Engine created. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get('projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688')


## Generating Memories

Memories will be generated from your conversation history stored in the `Session` object. The ADK `add_session_to_memory` method is responsible for triggering memory generation for your memory service, but the exact behavior depends on which service you use:

* `InMemoryMemoryService` persists all information, including the raw dialogue.

* `VertexAiMemoryBankService` only persists *meaningful* information. Not all conversations will result in generated memories. Additionally, the information will be condensed to individual, self-contained memories.

In this example, we create two sessions:
1. `chit_chat_session`, which is a conversation that contains events that may not be meaningful for future interactions.
2. `recommendations_session`, which is a conversation about looking for birthday presents for a family member. It includes implicit information that may be helpful for future conversations.

In [106]:
USER_ID = "My User"

chit_chat_session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)

call_agent(
    "what types of questions do you answer", chit_chat_session.id, user_id=USER_ID
)
# Refresh session object.
chit_chat_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=chit_chat_session.id
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Contents:
{"parts":[{"text":"what types of questions do you answer"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
As a large language model, I can answer a wide variety of questions across many domains. Here are some general categories of questions I can assi

Agent Response:  As a large language model, I can answer a wide variety of questions across many domains. Here are some general categories of questions I can assist with:

1.  **Factual Recall and Information Retrieval:**
    *   "What is the capital of France?"
    *   "Who wrote 'Romeo and Juliet'?"
    *   "When was the internet invented?"
    *   "What are the symptoms of a cold?"

2.  **Explanations and Concepts:**
    *   "How does photosynthesis work?"
    *   "Explain the theory of relativity."
    *   "What is blockchain technology?"
    *   "Define artificial intelligence."

3.  **Problem-Solving and Logic:**
    *   "How can I fix a leaky faucet?" (general steps, not specific diagnostics)
    *   "What's the best way to organize my digital files?"
    *   "Calculate the area of a circle with a radius of 5."

4.  **Creative Writing and Brainstorming:**
    *   "Write a short story about a talking cat."
    *   "Generate ideas for a marketing campaign for a new coffee shop."
 

In [107]:
recommendations_session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)

call_agent(
    "I have a three year old niece. What recommendations do you have for birthday presents?",
    recommendations_session.id,
    user_id=USER_ID,
)
call_agent(
    "I like the idea of a bike. Are there any age appropriate options?",
    recommendations_session.id,
    user_id=USER_ID,
)

recommendations_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=recommendations_session.id
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Contents:
{"parts":[{"text":"I have a three year old niece. What recommendations do you have for birthday presents?"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
That's wonderful! Three-year-olds are at such a fun age for gifts, as they're developing so many

Agent Response:  That's wonderful! Three-year-olds are at such a fun age for gifts, as they're developing so many new skills and their imaginations are really blossoming.

Here are some recommendations for your three-year-old niece, broken down by type:

**1. Creative & Art Supplies:**
*   **Washable Art Supplies:** Think chunky crayons, markers, finger paints, or even an easel with paper rolls. Washable is key!
*   **Play-Doh Sets:** These are always a hit, promoting fine motor skills and imaginative play. Look for sets with tools like rollers and cutters.
*   **Sticker Books:** Reusable sticker books or large sticker scene books can keep them engaged for a while.
*   **Simple Craft Kits:** Kits designed for toddlers, like mosaic sticker art or chunky bead threading.

**2. Building & Construction:**
*   **DUPLO Blocks:** These are perfect for this age – large enough to be safe, easy to connect, and allow for endless creative building.
*   **Magnetic Tiles (e.g., Magna-Tiles, PicassoTi

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
That's a fantastic idea! A bike is a wonderful gift that encourages outdoor play, develops coordination, and builds confidence. For a three-year-old, there are definitely age-appropriate options that are much better than trying to teach them on a traditional two-wheeler with training wheels right away.

Here are the best age-appropriate options for a three-year-old:

1.  **Balance Bikes (Highly Recommended!)**
    *   **What they are:** A balance bike is a two-wheeled bike without pedals. The child pushes off the ground with their feet to propel themselves forward, essentially learning to balance and steer before they learn to pedal.
    *   **Why they're great for 3-year-olds:**
        *   **Teaches Balance First:** This is the most crucial skill for riding a bike, and balance bike

Agent Response:  That's a fantastic idea! A bike is a wonderful gift that encourages outdoor play, develops coordination, and builds confidence. For a three-year-old, there are definitely age-appropriate options that are much better than trying to teach them on a traditional two-wheeler with training wheels right away.

Here are the best age-appropriate options for a three-year-old:

1.  **Balance Bikes (Highly Recommended!)**
    *   **What they are:** A balance bike is a two-wheeled bike without pedals. The child pushes off the ground with their feet to propel themselves forward, essentially learning to balance and steer before they learn to pedal.
    *   **Why they're great for 3-year-olds:**
        *   **Teaches Balance First:** This is the most crucial skill for riding a bike, and balance bikes teach it intuitively and quickly. Kids learn to glide with their feet up.
        *   **Builds Confidence:** Without the added complexity of pedals, children feel more in control and less

### InMemoryMemoryService

First, let's see what memories are extracted from the conversation using `InMemoryMemoryService`

The memories include the raw dialogue of the conversation.

In [108]:
await in_memory_service.add_session_to_memory(chit_chat_session)
await in_memory_service.add_session_to_memory(recommendations_session)

In [109]:
# Peek at the memories.
in_memory_service._session_events

{'memory_example_app/My User': {'062888a0-1c8b-42b8-9603-14a1f1ce9176': [Event(content=Content(
     parts=[
       Part(
         text='what types of questions do you answer'
       ),
     ],
     role='user'
   ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-e3d5b959-0347-4857-a8b8-a877be360b68', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None), long_running_tool_ids=None, branch=None, id='592196b9-c1a6-4da3-bb6f-83051133dc7c', timestamp=1761069812.202061),
   Event(c

In [110]:
await in_memory_service.search_memory(app_name=APP_NAME, user_id=USER_ID, query="test")

SearchMemoryResponse(memories=[])

In [111]:
await in_memory_service.search_memory(
    app_name=APP_NAME,
    user_id=USER_ID,
    query="what should I get my mom for mother's day",
)

SearchMemoryResponse(memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='what types of questions do you answer'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-21T18:03:32.202061'), MemoryEntry(content=Content(
  parts=[
    Part(
      text="""As a large language model, I can answer a wide variety of questions across many domains. Here are some general categories of questions I can assist with:

1.  **Factual Recall and Information Retrieval:**
    *   "What is the capital of France?"
    *   "Who wrote 'Romeo and Juliet'?"
    *   "When was the internet invented?"
    *   "What are the symptoms of a cold?"

2.  **Explanations and Concepts:**
    *   "How does photosynthesis work?"
    *   "Explain the theory of relativity."
    *   "What is blockchain technology?"
    *   "Define artificial intelligence."

3.  **Problem-Solving and Logic:**
    *   "How can I fix a leaky faucet?" (general steps, not specific diagnostics)
    *   "What's the best way 

### Memory Bank via `VertexAiMemoryBankService`

Now, let's use Memory Bank to generate and store memories. Unlike `InMemoryMemoryService`, only content that's meaningful interactions will be persisted. If there's no meaningful content, the `add_session_to_memory` call will be a no-op; memories are not generated.

Memory generation happens in the background. It's a non-blocking operation, so `add_session_to_memory` may complete before memories are actually generated in the background. If you want memory generation to be a blocking operation, use the Agent Engine SDK:

```
client.agent_engines.memories.generate(
    ...,
    config={
        # Default value is True.
        "wait_for_completion": True
    }
)
```

There are two key operations that Memory Bank performs under-the-hood with memory generation:

* **Extraction**: Extracts information about the user from their conversations with the agent. Only information that matches at least one of your instance's memory topics will be persisted.
* **Consolidation**: Identifies if existing memories with the same scope should be deleted or updated based on the extracted information. Memory Bank checks that new memories are not duplicative or contradictory before merging them with existing memories. If existing memories don't overlap with the new information, a new memory will be created.

#### ADK `add_session_to_memory`

In [112]:
await memory_bank_service.add_session_to_memory(chit_chat_session)
await memory_bank_service.add_session_to_memory(recommendations_session)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/58159578084278272' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-21T18:03:54.113447Z', 'updateTime': '2025-10-21T18:03:54.113447Z'}} done=None error=None response=None
INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/6493803445596717056' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {

If you run this immediately after `add_session_to_memory`, there may be no memories printed. `add_session_to_memory` is a non-blocking function and memory generation can take a few seconds to run.

In [113]:
await memory_bank_service.search_memory(
    app_name=APP_NAME,
    user_id=USER_ID,
    query="What should I get my mom for mother's day",
)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.


SearchMemoryResponse(memories=[])

#### Agent Engine SDK `GenerateMemories`

If I want to interact directly with Memory Bank, I can use the Agent Engine SDK. For example, I can use [`RetrieveMemories`](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/fetch-memories#scope-based) to retrieve my memories with or without similarity search.

Memories are isolated by their `scope`, which is an arbitrary dictionary. `VertexAiMemoryBankService` uses the scope keys `app_name` and `user_id`. `app_name` is set by your Runner, and `user_id` is set when creating your `Session`.

In [114]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME,
           "user_id": USER_ID},
)
list(response)

[]

You can also use `Retrievememories` to retrieve memories using similarity search.

In [115]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    similarity_search_params={
        "search_query": "what should I get my niece for her birthday?"
    },
)
list(response)

[]

Memory Bank uses a process called "consolidation" to ensure that we do not upload duplicative or contradictory information for the same scope.

In [116]:
import pprint

operation = client.agent_engines.memories.generate(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [
                        {
                            "text": "I like the idea of getting my niece a bike for her third birthday"
                        }
                    ],
                }
            }
        ]
    },
)
print("***OPERATION RESPONSE***")
pprint.pprint(operation)

print("\n***GENERATED MEMORIES***")
for generated_memory in operation.response.generated_memories:
    pprint.pprint(client.agent_engines.memories.get(name=generated_memory.memory.name))

***OPERATION RESPONSE***
AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-21T18:03:56.607877Z',
      'updateTime': '2025-10-21T18:04:03.876750Z'
    }
  },
  name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/8799646454810411008',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.UPDATED: 'UPDATED'>,
        memory=Memory(
          name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/memories/2013297599678775296'
        )
      ),
    ]
  )
)

***GENERATED MEMORIES***
Memory(
  create_time=datetime.datetime(2025, 10, 21, 18, 3, 57, 33102, tzinfo=TzInfo(UTC)),
  fact='I am considering a bike as a birthday present fo

What if your agent extracted memories that you want to combine with your existing memories? You can also use `GenerateMemories` to consolidate pre-extracted facts for the same scope [documenation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/generate-memories#consolidate-pre-extracted-memories).

Memory Bank already contains memories on this topic. So, the relevant existing memories will be updated rather than creating a new, duplicative memory.

In [117]:
operation = client.agent_engines.memories.generate(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    direct_memories_source={"direct_memories": [{"fact": "I got my niece a red bike"}]},
)

print("***OPERATION RESPONSE***")
pprint.pprint(operation)

print("\n***GENERATED MEMORIES***")
for generated_memory in operation.response.generated_memories:
    if generated_memory.action != "DELETED":
        pprint.pprint(
            client.agent_engines.memories.get(name=generated_memory.memory.name)
        )

***OPERATION RESPONSE***
AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-21T18:04:04.493716Z',
      'updateTime': '2025-10-21T18:04:09.441114Z'
    }
  },
  name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/7934955326355275776',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.UPDATED: 'UPDATED'>,
        memory=Memory(
          name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/memories/2013297599678775296'
        )
      ),
    ]
  )
)

***GENERATED MEMORIES***
Memory(
  create_time=datetime.datetime(2025, 10, 21, 18, 3, 57, 33102, tzinfo=TzInfo(UTC)),
  fact='I got my three-year-old niece a red bike as a bi

#### Automate with Callbacks

To automate the triggering of memory generation, you can implement a callback to generate memories after every turn. See the [`Remember this: Agent state and memory with ADK` blog post](https://cloud.google.com/blog/topics/developers-practitioners/remember-this-agent-state-and-memory-with-adk) for more information.

In [118]:
from google import adk


async def auto_save_session_to_memory_callback(callback_context):
    # Use the invocation context to access the conversation history that should
    # be used as the data source for memory generation.
    await memory_bank_service.add_session_to_memory(
        callback_context._invocation_context.session
    )
    print("\n****Triggered memory generation****\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    after_agent_callback=auto_save_session_to_memory_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [119]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "What christmas present recommendations do you have for my niece?",
    session.id,
    user_id=USER_ID,
)

call_agent(
    "I think I'll get her a doll. What types are good for her age?",
    session.id,
    user_id=USER_ID,
)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.898612030326246 memory=Memory(
  create_time=datetime.datetime(2025, 10, 21, 18, 3, 57, 33102, tzinfo=TzInfo(UTC)),
  fact='I got my three-year-old niece a red bike as a birthday present.',
  name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/memories/2013297599678775296',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 21, 18, 4, 9, 358398, tzinfo=TzInfo(UTC))
)
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your 

Agent Response:  That's wonderful! Since your niece just turned three and received a red bike for her birthday, here are some Christmas present recommendations that would complement her current interests and developmental stage:

1.  **Imaginative Play Sets:**
    *   **Play Kitchen or Food Set:** Three-year-olds love to mimic adults. A small play kitchen with toy food, pots, and pans encourages creativity and role-playing.
    *   **Dollhouse or Animal Figures:** Simple dollhouses or sets of animal figures (like farm animals or forest creatures) can spark endless storytelling.
    *   **Dress-Up Clothes:** A box of fun costumes (e.g., princess, superhero, doctor, animal) can fuel hours of imaginative adventures.

2.  **Building and Construction Toys:**
    *   **LEGO Duplo Set:** These larger bricks are perfect for little hands to build and create without frustration. Many sets come with figures, animals, or vehicles.
    *   **Magnetic Tiles (e.g., Magna-Tiles, Connetix):** These are

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/6759515823611576320' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-21T18:04:18.607693Z', 'updateTime': '2025-10-21T18:04:18.607693Z'}} done=None error=None response=None



****Triggered memory generation****



INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Contents:
{"parts":[{"text":"What christmas present recommendations do you have for my niece?"}],"role":"user"}
{"parts":[{"text":"That's wonderful! Since your niece just turned three and received a red bike for her birthday, here are some Christmas present recommendations that would complement her current interests and developmental stage:\n\n1.  **Imaginative Play Sets:**\n    *   **Play Kitchen or Food Set:** Three-year-olds love to mimic adults. A small play kitchen with toy food, pots, and pans encourages creativity and role-playing.\n    *   **Dollhouse or 

Agent Response:  That's a lovely idea! Dolls are fantastic for encouraging nurturing play, empathy, storytelling, and developing fine motor skills through dressing and undressing. For a three-year-old, here are some great types of dolls to consider:

1.  **Baby Dolls (Classic & Wonderful):**
    *   **Why they're great:** Three-year-olds love to mimic what they see, and caring for a baby doll is a natural extension of that. They can feed, burp, change, and comfort their "baby," which fosters empathy and imaginative play.
    *   **What to look for:**
        *   **Soft body:** Many baby dolls have a soft, huggable body, making them perfect for cuddling.
        *   **Washable:** Kids this age can be messy, so a doll that can be wiped clean or even machine-washed (check tags!) is a huge plus.
        *   **Simple clothing:** Look for outfits that are easy for little hands to put on and take off (e.g., Velcro closures, elastic waistbands, larger buttons/snaps).
        *   **No small par

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/1023088879905/locations/us-central1/reasoningEngines/5339215270340722688/operations/6241601866463969280' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-21T18:04:38.581571Z', 'updateTime': '2025-10-21T18:04:38.581571Z'}} done=None error=None response=None



****Triggered memory generation****



In [120]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    similarity_search_params={
        "search_query": "What was I thinking about getting my niece for Christmas?"
    },
)
list(response)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Failed to generate embeddings: Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.', 'status': 'RESOURCE_EXHAUSTED'}}

The prior example used the entire Session to generate memories. However, this can be costly as the size of the session increases.

You can also use callbacks to only save the current turn to Memory Bank. This has the downside, however, where you lose some of the context of the conversation.

In [ ]:
async def auto_save_user_turn_to_memory_callback(callback_context):
    last_turn = callback_context._invocation_context.user_content
    client.agent_engines.memories.generate(
        name=agent_engine.api_resource.name,
        scope={"app_name": APP_NAME, "user_id": USER_ID},
        direct_contents_source={"events": [{"content": last_turn}]},
        config={"wait_for_completion": False},
    )
    print("\n****Triggered memory generation****\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    before_agent_callback=auto_save_user_turn_to_memory_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [ ]:
call_agent(
    "I think I'll get her a doll. What types are good for her age?",
    session.id,
    user_id=USER_ID,
)

## Using Memory in your agent

Now, let's link our agent to Memory, so that memories can be fetched by your agent and used for inference. This allows your agent to remember information from prior sessions in a new, empty session.

You can use ADK's built-in tools to fetch memories and incorporate them in the prompt. When using the built-in tools, it's important to provide both a memory tool when creating your Agent and a memory service when defining your Runner.

### `adk.PreloadMemoryTool`
The `PreloadMemoryTool` always retrieves memories at the start of each turn and includes the memories in the system instruction.

Under the hood, it invokes `memory_service.search_memory` for the given User and App Name. Memory is invoked before the LLM is called, so there will not be an associated tool call logged for memory.

We're going to use a [callback](https://google.github.io/adk-docs/callbacks/) to print out the System Instructions. Since we use `before_model_callback`, the callback executes right before the model is called.

<img src="https://google.github.io/adk-docs/assets/callback_flow.png">

In [ ]:
from google import adk


def log_system_instructions(callback_context, llm_request):
    """A callback to print the LLM request."""
    print(
        f"\n*System Instruction*:\n{llm_request.config.system_instruction}\n*********\n"
    )


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    before_model_callback=log_system_instructions,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

This conversation uses a new session, so it doesn't have access to the user's conversation history without using memory.

Since we're using `PreloadMemoryTool`, the retrieved memories will be appended to the System Instructions.

In [ ]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "Can you remind me what I got my niece for her christmas?",
    session.id,
    user_id=USER_ID,
)

### adk.LoadMemoryTool

Now, let's use `LoadMemoryTool`. Unlike `PreloadMemoryTool`, this tool acts like a standard tool. Your agent needs to decide whether the tool should be invoked.

In [ ]:
from google import adk


def log_tool_call(tool, args, tool_response, **kwargs):
    """A callback to print the LLM request."""
    print(f"\n*Tool*:\n{tool}")
    print(f"\n*Tool call*:\n{args}")
    print(f"\n*Tool response*:\n{tool_response}\n*********\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.load_memory_tool.LoadMemoryTool()],
    before_model_callback=log_system_instructions,
    after_tool_callback=log_tool_call,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [ ]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "Can you remind me what I got my niece for her christmas?",
    session.id,
    user_id=USER_ID,
)

If the agent decides that Memory is not helpful for a given query, the Memory tool will not be invoked.

In [ ]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent("Hi!", session.id, user_id=USER_ID)

### Custom callback to retrieve memories

If you want more control over how memories are retrieved, you can create your own custom callback.

For example, you can use the Agent Engine SDK within the callback to retrieve memories without similarity search. This is lower latency than using similarity search.

You can also customize your `scope` rather than using the default "app_name" and "user_id" scope keys used by ADK's memory service.

In [ ]:
def retrieve_memories_callback(callback_context, llm_request):
    user_id = callback_context._invocation_context.user_id

    response = client.agent_engines.memories.retrieve(
        name=agent_engine.api_resource.name,
        # Unlike the ADK Memory Service, this does not use App Name in the scope.
        scope={"user_id": user_id},
    )
    memories = [f"* {memory.memory.fact}" for memory in list(response)]
    if not memories:
        # No memories to add to System Instructions.
        return
    # Append formatted memories to the System Instructions
    llm_request.config.system_instruction += (
        "\nHere is information that you have about the user:\n"
    )
    llm_request.config.system_instruction += "\n".join(memories)


def log_tool_call(tool, args, tool_response, **kwargs):
    """A callback to print the LLM request."""
    print(f"\n*Tool*:\n{tool}")
    print(f"\n*Tool call*:\n{args}")
    print(f"\n*Tool response*:\n{log_tool_call}\n*********\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    before_model_callback=retrieve_memories_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

Since we're using a different scope than the ADK Memory Service, we don't have access to the prior memories. Memories are isolated by their scope dictionary.

In [ ]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent("Hi!", session.id, user_id=USER_ID)

Let's first generate memories for our new scope.

In [ ]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I have four nieces!"}]}}
        ]
    },
)

In [ ]:
call_agent("What information do you know about me?", session.id, user_id=USER_ID)

## Customize your Memory Bank's behavior

So far, we've used Memory Bank with it's default settings. Now, let's customize some configurations. The configuration is set when creating or updating your Agent Engine.

### Customizing Memory Extraction

By default, Memory Bank considers memories that fit the following topics to be meaningful:

* **Personal information (`USER_PERSONAL_INFO`)**: Significant personal information about the user, like names, relationships, hobbies, and important dates. For example, "I work at Google" or "My wedding anniversary is on December 31".
* **User preferences (`USER_PREFERENCES`)**: Stated or implied likes, dislikes, preferred styles, or patterns. For example, "I prefer the middle seat."
* **Key conversation events and task outcomes (`KEY_CONVERSATION_DETAILS`)**: Important milestones or conclusions within the dialogue. For example, "I booked plane tickets for a round trip between JFK and SFO. I leave on June 1, 2025 and return on June 7, 2025."
* **Explicit remember / forget instructions (`EXPLICIT_INSTRUCTIONS`)**: Information that the user explicitly asks the agent to remember or forget. For example, if the user says "Remember that I primarily use Python," Memory Bank generates a memory such as "I primarily use Python."

These are managed topics where Memory Bank manages their definition. If you want to use a subset of these topics or use custom topics, you can provide a "CustomizationConfig" to customize what information Memory Bank should consider meaningful to persist.

#### Managed Topics

We're going to update our Memory Bank to only extract user preferences.

This will take affect for all requests to Memory Bank, regardless of whether you use ADK or Agent Engine SDK.

In [ ]:
user_preferences_config = {
    "scope_keys": ["user_id"],
    "memory_topics": [
        {"managed_memory_topic": {"managed_topic_enum": "USER_PREFERENCES"}}
    ],
}


client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "customization_configs": [user_preferences_config],
            }
        }
    },
)

Memories that fit the `USER_PREFERENCES` category will be persisted.

In [ ]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I prefer the aisle seat"}],
                }
            }
        ]
    },
)

Memories that don't fit `USER_PREFERENCES` won't be persisted.

In [ ]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I'm going to get my niece a red bike"}],
                }
            }
        ]
    },
)

#### Custom Topics
Alternatively, you can define your own custom topics where you define your own labels, descriptions, and examples.

In [ ]:
from vertexai.types import MemoryBankCustomizationConfig as CustomizationConfig
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExample as GenerateMemoriesExample,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleConversationSource as ConversationSource,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleConversationSourceEvent as ConversationSourceEvent,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleGeneratedMemory as ExampleGeneratedMemory,
)
from vertexai.types import MemoryBankCustomizationConfigMemoryTopic as MemoryTopic
from vertexai.types import (
    MemoryBankCustomizationConfigMemoryTopicCustomMemoryTopic as CustomMemoryTopic,
)

memory_topic = MemoryTopic(
    custom_memory_topic=CustomMemoryTopic(
        label="business_feedback",
        description="""Specific user feedback about their experience at
the coffee shop. This includes opinions on drinks, food, pastries, ambiance,
staff friendliness, service speed, cleanliness, and any suggestions for
improvement.""",
    )
)


example = GenerateMemoriesExample(
    conversation_source=ConversationSource(
        events=[
            ConversationSourceEvent(
                content=Content(
                    role="model",
                    parts=[
                        Part(
                            text="Welcome back to The Daily Grind! We'd love to hear your feedback on your visit."
                        )
                    ],
                )
            ),
            ConversationSourceEvent(
                content=Content(
                    role="user",
                    parts=[
                        Part(
                            text="Hey. The drip coffee was a bit lukewarm today, which was a bummer. Also, the music was way too loud, I could barely hear my friend."
                        )
                    ],
                )
            ),
        ]
    ),
    generated_memories=[
        ExampleGeneratedMemory(
            fact="The user reported that the drip coffee was lukewarm."
        ),
        ExampleGeneratedMemory(
            fact="The user felt the music in the shop was too loud."
        ),
    ],
)

noop_example = GenerateMemoriesExample(
    conversation_source=ConversationSource(
        events=[
            ConversationSourceEvent(
                content=Content(
                    role="model",
                    parts=[
                        Part(
                            text="Welcome back to The Daily Grind! We'd love to hear your feedback on your visit."
                        )
                    ],
                )
            ),
            ConversationSourceEvent(
                content=Content(
                    role="user", parts=[Part(text="Thanks for the coffee!")]
                )
            ),
        ]
    ),
    generated_memories=[],
)

client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "customization_configs": [
                    CustomizationConfig(
                        memory_topics=[memory_topic],
                        generate_memories_examples=[example, noop_example],
                    )
                ],
            }
        }
    },
)

In [ ]:
client.agent_engines.memories.generate(
    scope={"user_id": "123"},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I prefer the aisle seat"}],
                }
            }
        ]
    },
)

In [ ]:
import vertexai

client.agent_engines.memories.generate(
    scope={"user_id": "123"},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "You should have almond milk"}],
                }
            }
        ]
    },
)

In [ ]:
list(
    client.agent_engines.memories.retrieve(
        name=agent_engine.api_resource.name, scope={"user_id": "123"}
    )
)

### Dynamic TTL

You can also configure Memory Bank to automatically set the TTL for any generated or manually created (or updated) memories.

#### Default TTL

Default TTL applies to all operations that create or update a Memory.

In [ ]:
client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "ttl_config": {
                    # 30 days
                    "default_ttl": f"{60 * 60 * 24 * 30}s"
                }
            }
        }
    },
)

In [ ]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I have four nieces!"}]}}
        ]
    },
    config={"wait_for_completion": True},
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

#### Granular TTL

The TTL is calculated based on which operation created or updated the Memory. If not set for a given operation, then the operation won't update the Memory's expiration time.

Parameters include:
 * `create_ttl`: Applies to Memories created with CreateMemory operations
 * `generate_created_ttl`: Applies to Memories created with GenerateMemories
 * `generate_updated_ttl`: Applies to Memories updated with GenerateMemories

In [ ]:
client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "ttl_config": {
                    "granular_ttl_config": {
                        # 365 days
                        "generate_created_ttl": f"{24 * 60 * 60 * 365}s",
                    }
                }
            }
        }
    },
)

In [ ]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "Remember that I work at Zindua School!"}],
                }
            }
        ]
    },
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

In [ ]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I work at Google!"}]}}
        ]
    },
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

## Cleaning up

It's always a best practice in cloud development to clean up resources you no longer need to avoid incurring unexpected costs. This final cell deletes the AgentEngine resources we created throughout this tutorial.

In [ ]:
delete_agent_engines = True

agent_engine_name = agent_engine.api_resource.name

if delete_agent_engines:
    # Delete agent engine
    client.agent_engines.delete(name=agent_engine_name, force=True)